# EXP_001 — Electrophysiological Recording

Plotting ADC-24 differential recording from *Pleurotus eryngii* mycelium.

**Reference:** Mishra et al., *Sci. Robot.* 2024 — expected spontaneous spikes ~135 µV mean, up to ~1868 µV.

In [19]:
import h5py
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots

## 1. Load data

In [20]:
HDF5_PATH = "faraday cage.hdf5"
SAMPLE_RATE = 10  # S/s from ADC-24 config

with h5py.File(HDF5_PATH, "r") as f:
    print("Datasets:", list(f.keys()))
    raw_volts = f["Differential 1 - 2"][:]

# Convert to millivolts and microvolts
raw_mv = raw_volts * 1e3
raw_uv = raw_volts * 1e6

# Time axis
n_samples = len(raw_volts)
t = np.arange(n_samples) / SAMPLE_RATE

print(f"Samples: {n_samples}")
print(f"Duration: {t[-1]:.1f} s ({t[-1]/60:.1f} min)")
print(f"Mean: {np.mean(raw_mv):.4f} mV ({np.mean(raw_uv):.1f} µV)")
print(f"Std:  {np.std(raw_mv):.4f} mV ({np.std(raw_uv):.1f} µV)")
print(f"Range: {np.min(raw_mv):.4f} to {np.max(raw_mv):.4f} mV")

Datasets: ['Differential 1 - 2']
Samples: 1571
Duration: 157.0 s (2.6 min)
Mean: 0.0967 mV (96.7 µV)
Std:  0.0272 mV (27.2 µV)
Range: -0.2990 to 0.3833 mV


## 2. Raw signal — full trace

In [21]:
fig = go.Figure()

fig.add_trace(go.Scatter(
    x=t, y=raw_mv,
    mode="lines",
    name="Differential 1-2",
    line=dict(color="#00d4aa", width=1),
))

fig.update_layout(
    title="EXP_001 — Raw ADC-24 Recording",
    xaxis_title="Time (s)",
    yaxis_title="Voltage (mV)",
    template="plotly_dark",
    height=500,
    hovermode="x unified",
    xaxis=dict(rangeslider=dict(visible=True)),
)

fig.show()

## 3. Signal in microvolts (baseline-subtracted)

Subtract the DC offset to see deviations from baseline — this is where spikes would appear.

In [22]:
baseline = np.mean(raw_uv)
centered_uv = raw_uv - baseline

fig2 = go.Figure()

fig2.add_trace(go.Scatter(
    x=t, y=centered_uv,
    mode="lines",
    name="Centered signal",
    line=dict(color="#ff6b6b", width=1),
))

# Reference lines for expected spike thresholds
fig2.add_hline(y=135, line_dash="dash", line_color="#ffd43b",
               annotation_text="Mean spike (Mishra: 135 µV)", annotation_position="top left")
fig2.add_hline(y=-135, line_dash="dash", line_color="#ffd43b")
fig2.add_hline(y=10, line_dash="dot", line_color="#868e96",
               annotation_text="Peak detection threshold (10 µV)", annotation_position="bottom left")

fig2.update_layout(
    title="EXP_001 — Baseline-Subtracted Signal",
    xaxis_title="Time (s)",
    yaxis_title="Voltage (µV)",
    template="plotly_dark",
    height=500,
    hovermode="x unified",
)

fig2.show()

## 4. Signal statistics

In [23]:
fig3 = go.Figure()

fig3.add_trace(go.Histogram(
    x=centered_uv,
    nbinsx=50,
    marker_color="#00d4aa",
    opacity=0.8,
    name="Amplitude distribution",
))

fig3.update_layout(
    title="EXP_001 — Amplitude Distribution (baseline-subtracted)",
    xaxis_title="Voltage (µV)",
    yaxis_title="Count",
    template="plotly_dark",
    height=400,
)

fig3.show()